In [7]:
import numpy as np
import math

def calcular_factorial(n):
    """Función auxiliar para calcular factoriales."""
    return math.factorial(n)

def polinomio_radial(n, m, rho):
    """
    Calcula el componente radial R_nm(rho) del polinomio de Zernike.
    Si n-|m| es impar, el polinomio es 0.
    """
    if (n - abs(m)) % 2 != 0:
        return 0
    
    R = 0
    limite_s = (n - abs(m)) // 2
    
    for s in range(limite_s + 1):
        numerador = ((-1)**s) * calcular_factorial(n - s)
        denominador = (calcular_factorial(s) * calcular_factorial((n + abs(m)) // 2 - s) * calcular_factorial((n - abs(m)) // 2 - s))
        R += (numerador / denominador) * (rho ** (n - 2*s))
    
    return R

def calcular_momentos_zernike(imagen, n, m, nombre_ejercicio="Ejercicio"):
    """
    Calcula el momento de Zernike Anm paso a paso imprimiendo todo.
    """
    filas, cols = imagen.shape
    
    print(f"\n{'='*80}")
    print(f"PROCESANDO: {nombre_ejercicio}")
    print(f"Parámetros: n={n}, m={m}")
    print(f"{'='*80}")
    
    # 1. Calcular Masa (m00)
    masa = np.sum(imagen)
    print(f"1. Masa total (suma de 1s): {masa}")
    
    if masa == 0:
        print("La imagen está vacía.")
        return 0
    
    # 2. Calcular Centroide (cy, cx)
    y_indices, x_indices = np.indices((filas, cols))
    
    m10 = np.sum(x_indices * imagen) # Momento en x
    m01 = np.sum(y_indices * imagen) # Momento en y
    
    cx = m10 / masa
    cy = m01 / masa
    
    print(f"2. Centro Geométrico (cy, cx): ({cy:.4f}, {cx:.4f})")
    
    # 3. Calcular Radio Máximo (R_max)
    dist_sq_00 = (0 - cy)**2 + (0 - cx)**2
    r_max = np.sqrt(dist_sq_00)
    print(f"3. Radio Máximo (R_max) desde el centro a (0,0): {r_max:.4f}")
    
    # 4. Sumatoria Zernike
    suma_zernike = 0 + 0j
    
    # Obtenemos las coordenadas de los píxeles activos
    puntos_y, puntos_x = np.where(imagen == 1)
    
    print("\n--- DETALLE DE CÁLCULO POR PÍXEL (TODOS LOS PUNTOS) ---")
    # Cabecera para entender las columnas
    print(f"{'Píxel':<12} | {'y_cent':<8} | {'x_cent':<8} | {'rho':<6} | {'R_nm':<7} | {'theta':<7} | {'Angular':<18} | {'Term (R*Ang)':<18}")
    print("-" * 110)

    for i, (y, x) in enumerate(zip(puntos_y, puntos_x)):
        # Coordenadas centradas
        y_cent = y - cy
        x_cent = x - cx
        
        # Coordenadas polares
        r_pixel = np.sqrt(y_cent**2 + x_cent**2)
        rho = r_pixel / r_max # Normalización
        
        # Ángulo theta
        theta = np.arctan2(y_cent, x_cent)
        
        # Polinomio Radial R_nm
        R_val = polinomio_radial(n, m, rho)
        
        # Núcleo angular V_nm* = exp(-j * m * theta)
        angular_val = np.exp(-1j * m * theta)
        
        term = R_val * angular_val
        suma_zernike += term
        
        # Impresión formateada "tal cual" pediste
        print(f"({y},{x})".ljust(12) + 
              f" | {y_cent:>8.4f} | {x_cent:>8.4f} | {rho:.4f} | {R_val:>7.4f} | {theta:>7.4f} | {angular_val:.4f} | {term:.4f}")

    # Factor de normalización (n+1)/pi
    factor_normalizacion = (n + 1) / np.pi
    A_nm = factor_normalizacion * suma_zernike
    
    print("-" * 110)
    print(f"\n--- RESULTADO FINAL A_{n}{m} ---")
    print(f"Suma acumulada (Sigma): {suma_zernike:.4f}")
    print(f"Factor normalización ((n+1)/pi): {factor_normalizacion:.4f}")
    print(f"Valor Momento A_{n}{m}: {A_nm:.4f}")
    print(f"Magnitud |A_{n}{m}|: {abs(A_nm):.4f}")
    
    return A_nm

# --- DEFINICIÓN DE MATRICES ---

# Ejercicio 1: Matriz 5x5
imagen_1 = np.array([
    [0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 0, 1] 
])

# Ejercicio 2: Matriz 9x9
imagen_2 = np.array([
    [0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
])

# --- EJECUCIÓN ---

# Ejercicio 1 (Letra A)
calcular_momentos_zernike(imagen_1, n=2, m=0, nombre_ejercicio="Ej 1 - Caso A (5x5)")
calcular_momentos_zernike(imagen_1, n=5, m=3, nombre_ejercicio="Ej 1 - Caso B (5x5)")

# Ejercicio 2 (Número 3)
calcular_momentos_zernike(imagen_2, n=2, m=0, nombre_ejercicio="Ej 2 - Caso A (9x9)")
calcular_momentos_zernike(imagen_2, n=5, m=3, nombre_ejercicio="Ej 2 - Caso B (9x9)")


PROCESANDO: Ej 1 - Caso A (5x5)
Parámetros: n=2, m=0
1. Masa total (suma de 1s): 10
2. Centro Geométrico (cy, cx): (2.2000, 2.1000)
3. Radio Máximo (R_max) desde el centro a (0,0): 3.0414

--- DETALLE DE CÁLCULO POR PÍXEL (TODOS LOS PUNTOS) ---
Píxel        | y_cent   | x_cent   | rho    | R_nm    | theta   | Angular            | Term (R*Ang)      
--------------------------------------------------------------------------------------------------------------
(0,2)        |  -2.2000 |  -0.1000 | 0.7241 |  0.0486 | -1.6162 | 1.0000+0.0000j | 0.0486+0.0000j
(1,1)        |  -1.2000 |  -1.1000 | 0.5352 | -0.4270 | -2.3127 | 1.0000+0.0000j | -0.4270+0.0000j
(1,3)        |  -1.2000 |   0.9000 | 0.4932 | -0.5135 | -0.9273 | 1.0000+0.0000j | -0.5135+0.0000j
(2,1)        |  -0.2000 |  -1.1000 | 0.3676 | -0.7297 | -2.9617 | 1.0000+0.0000j | -0.7297+0.0000j
(2,2)        |  -0.2000 |  -0.1000 | 0.0735 | -0.9892 | -2.0344 | 1.0000+0.0000j | -0.9892+0.0000j
(2,3)        |  -0.2000 |   0.9000 | 0.3031

np.complex128(-1.4170841993100265+1.3363706460686706j)